In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets
from torchvision import transforms

import numpy as np
from tqdm import tqdm
import math

In [2]:
transform = transforms.Compose([
                    transforms.ToTensor(), 
                    # transforms.Normalize((0.5,), (0.5,)),
            ])

In [3]:
def CreateDataLoaders(Option, p_BatchSize):
    if Option == 1:
        train_dataset = datasets.MNIST("./", train=True, transform = transform, download=True)
        train_subset, val_subset = torch.utils.data.random_split(train_dataset, [int(len(train_dataset) * 0.85), int(len(train_dataset) * 0.15)], generator=torch.Generator().manual_seed(1))
        test_dataset = datasets.MNIST("./", train=False, transform = transform, download=True)
    elif Option == 2:
        train_dataset = datasets.FashionMNIST("./", train=True, transform = transform, download=True)
        train_subset, val_subset = torch.utils.data.random_split(train_dataset, [int(len(train_dataset) * 0.85), int(len(train_dataset) * 0.15)], generator=torch.Generator().manual_seed(1))
        test_dataset = datasets.FashionMNIST("./", train=False, transform = transform, download=True)
    
    elif Option == 3:
        train_dataset = datasets.STL10("./", split="train", transform = transform, download=True)
        train_subset, val_subset = torch.utils.data.random_split(train_dataset, [int(len(train_dataset) * 0.85), int(len(train_dataset) * 0.15)], generator=torch.Generator().manual_seed(1))
        test_dataset = datasets.STL10("./", split="test", transform = transform, download=True)
    
    elif Option == 4:
        train_dataset = datasets.SVHN("./", split="train", transform = transform, download=True)
        print(len(train_dataset))
        train_subset, val_subset = torch.utils.data.random_split(train_dataset, [int(len(train_dataset) * 0.85), math.ceil(len(train_dataset) * 0.15)], generator=torch.Generator().manual_seed(1))
        test_dataset = datasets.SVHN("./", split="test", transform = transform, download=True)
    
    elif Option == 5:
        train_dataset = datasets.CIFAR10("./", train=True, transform = transform, download=True)
        train_subset, val_subset = torch.utils.data.random_split(train_dataset, [int(len(train_dataset) * 0.85), int(len(train_dataset) * 0.15)], generator=torch.Generator().manual_seed(1))
        test_dataset = datasets.CIFAR10("./", train=False, transform = transform, download=True)

    elif Option == 6:
        train_dataset = datasets.CIFAR100("./", train=True, transform = transform, download=True)
        train_subset, val_subset = torch.utils.data.random_split(train_dataset, [int(len(train_dataset) * 0.85), int(len(train_dataset) * 0.15)], generator=torch.Generator().manual_seed(1))
        test_dataset = datasets.CIFAR100("./", train=False, transform = transform, download=True)

    
    Train_DataLoader = torch.utils.data.DataLoader(train_subset, batch_size = p_BatchSize, shuffle = True)
    Val_DataLoader = torch.utils.data.DataLoader(val_subset, batch_size = p_BatchSize, shuffle = True)
    Test_DataLoader = torch.utils.data.DataLoader(test_dataset, batch_size = p_BatchSize, shuffle = True)

    return Train_DataLoader, Val_DataLoader, Test_DataLoader

In [4]:
class MLP_IOCN(nn.Module):
    def __init__(self, InputDim, OutputDim):
        super(MLP_IOCN, self).__init__()
        self.InputDim = InputDim
        self.OutputDim = OutputDim
        self.Linear1 = nn.Linear(InputDim, 800)
        self.Linear2 = nn.Linear(800, 800)
        self.Linear3 = nn.Linear(800, self.OutputDim)
        # self.Linear4 = nn.Linear(800, self.OutputDim)
        self.ActFunc = nn.functional.elu
        self.batch = nn.BatchNorm1d(800)
        # self.SftMax = nn.Softmax

    def forward(self, x):
        x = self.ActFunc(self.batch(self.Linear1(x)))
        x = self.ActFunc(self.batch(self.Linear2(x)))
        # x = self.ActFunc(self.batch(self.Linear3(x)))

        # output = self.SftMax(self.Linear4(x))
        
        return self.Linear3(x)
    
    # def InitWeights(self):
    #     torch.nn.init.uniform_(self.Linear1.weight,-0.5, 0.5)
    #     torch.nn.init.uniform_(self.Linear2.weight,-0.5, 0.5)

In [5]:
model = MLP_IOCN(100, 10)
for name, param in model.named_parameters():
    print("name=", name)
    # print("param=", param)

name= Linear1.weight
name= Linear1.bias
name= Linear2.weight
name= Linear2.bias
name= Linear3.weight
name= Linear3.bias
name= batch.weight
name= batch.bias


In [6]:
def TrainModel(p_model, loss_criteria, Optimizer, device,  p_TrainDL, Gamma = 5):
    TrainCorr = 0
    TotNumOfSamples = 0

    for images, labels in tqdm(p_TrainDL):
        
        for name, param in p_model.named_parameters():
            # if "Linear1" in name or "bias" in name:
            if "Linear1" in name or "bias" in name:
                continue
            
            tmpParam = param.data
            NewTmpParam = torch.exp(tmpParam - Gamma)
            tmpParam = torch.where(tmpParam<0, NewTmpParam, tmpParam)
            param.data = tmpParam

            # temp_weight = param.data
            # # temp_weight[temp_weight < 0] = torch.clamp(temp_weight[temp_weight < 0], min=0)
            # temp_weight[temp_weight < 0] = torch.exp(temp_weight[temp_weight < 0])
            # param.data = temp_weight
            
        Optimizer.zero_grad()

        images = images.reshape(images.shape[0], -1).to(device)
        labels = labels.to(device)

        pred = p_model(images)
        # print(type(pred))
        # print(pred.shape)
        # print(pred)
        # pred=pred.to(torch.float)
        # labels=labels.to(torch.float)
        predClass = torch.max(pred.data, 1)[1]
        # predClass = np.argmax(pred, axis=-1)
        # predClass = pred.max(1)
        # print("Pred", predClass)
        # print(type(labels))
        # print("labels", labels)

        # predClass=predClass.to(torch.float)
        loss = loss_criteria(pred, labels)
        TrainCorr += (predClass==labels).sum()
        TotNumOfSamples += len(labels)

        loss.backward()
        Optimizer.step()
    
    return TrainCorr/TotNumOfSamples

def EvaluateModel(p_model, p_loader, device):
    Correct = 0
    TotalNumOfSamples = 0

    for images, labels in tqdm(p_loader):
        images = images.reshape(images.shape[0], -1).to(device)
        labels = labels.to(device)

        pred = p_model(images)
        predClass = torch.max(pred.data, 1)[1]
        Correct += (predClass==labels).sum()
        TotalNumOfSamples += len(labels)
        # loss = loss_criteria(predClass, labels)
        # Accuracy += loss.cpu().data.item()

    return Correct/TotalNumOfSamples

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 100

In [ ]:
# -------------------------------------------- MNIST Dataset -----------------------------------------------------------
Option = 1
Train_DataLoader, Val_DataLoader, Test_DataLoader = CreateDataLoaders(Option, 64)
ModelName = "Model_MLP_IOCN_MNIST.pt"

model = MLP_IOCN(784, 10).to(device)
loss_criteria = nn.CrossEntropyLoss()
AdamOpt = torch.optim.Adam(model.parameters(), lr=0.0001)

ValAccuracy = 0

saved = False
if saved == True:
    saved_model = torch.load(ModelName, map_location=torch.device('cpu')).to(device)
else:
    model.train()
    Train_Accuracy = 0

    for e in range(EPOCHS):
        Train_Accuracy = TrainModel(model, loss_criteria, AdamOpt, device, Train_DataLoader)
        val_acc = EvaluateModel(model, Val_DataLoader, device)

        print("EPOCH - ", e+1, ". Train Accuracy = ", Train_Accuracy.cpu().item(), ", Validation Accuracy = ", val_acc.cpu().item())
        
        if val_acc.cpu().item() > ValAccuracy:
            print("Model Re-Saved")
            ValAccuracy = val_acc.cpu().item()
            torch.save(model, ModelName)


    saved_model = torch.load(ModelName, map_location=torch.device('cpu')).to(device)

Train_Accuracy = EvaluateModel(saved_model, Train_DataLoader, device)
print("Train Accuracy = ", Train_Accuracy.cpu().item())
Test_Accuracy = EvaluateModel(saved_model, Test_DataLoader, device)
print("Test Accuracy = ", Test_Accuracy.cpu().item())

print("Generalization Gap = ", (Train_Accuracy.cpu().item() - Test_Accuracy.cpu().item()))

100%|██████████| 141/141 [00:01<00:00, 95.96it/s]


EPOCH -  1 . Train Accuracy =  0.8982353210449219 , Validation Accuracy =  0.9318888783454895
Model Re-Saved


100%|██████████| 141/141 [00:01<00:00, 109.57it/s]


EPOCH -  2 . Train Accuracy =  0.9434705972671509 , Validation Accuracy =  0.9486666917800903
Model Re-Saved


100%|██████████| 141/141 [00:01<00:00, 114.13it/s]


...

EPOCH -  100 . Train Accuracy =  0.9993921518325806 , Validation Accuracy =  0.9781111478805542


100%|██████████| 797/797 [00:07<00:00, 110.76it/s]


Train Accuracy =  0.999333381652832


100%|██████████| 157/157 [00:01<00:00, 110.57it/s]

Test Accuracy =  0.9781000018119812
Generalization Gap =  0.02123337984085083


In [ ]:
saved_model = torch.load(ModelName, map_location=torch.device('cpu')).to(device)

Train_Accuracy = EvaluateModel(saved_model, Train_DataLoader, device)
print("Train Accuracy = ", Train_Accuracy.cpu().item())
Test_Accuracy = EvaluateModel(saved_model, Test_DataLoader, device)
print("Test Accuracy = ", Test_Accuracy.cpu().item())

print("Generalization Gap = ", (Train_Accuracy.cpu().item() - Test_Accuracy.cpu().item()))

100%|██████████| 797/797 [00:07<00:00, 101.47it/s]


Train Accuracy =  0.9889608025550842


100%|██████████| 157/157 [00:01<00:00, 105.81it/s]

Test Accuracy =  0.9723999500274658
Generalization Gap =  0.016560852527618408


In [ ]:
# -------------------------------------------- FMNIST Dataset -----------------------------------------------------------
Option = 2
Train_DataLoader, Val_DataLoader, Test_DataLoader = CreateDataLoaders(Option, 64)
ModelName = "Model_MLP_IOCN_FMNIST.pt"

model = MLP_IOCN(784, 10).to(device)
loss_criteria = nn.CrossEntropyLoss()
AdamOpt = torch.optim.Adam(model.parameters(), lr=0.0001)

ValAccuracy = 0

saved = False
if saved == True:
    saved_model = torch.load(ModelName, map_location=torch.device('cpu')).to(device)
else:
    model.train()
    Train_Accuracy = 0

    for e in range(EPOCHS):
        Train_Accuracy = TrainModel(model, loss_criteria, AdamOpt, device, Train_DataLoader)
        val_acc = EvaluateModel(model, Val_DataLoader, device)

        print("EPOCH - ", e+1, ". Train Accuracy = ", Train_Accuracy.cpu().item(), ", Validation Accuracy = ", val_acc.cpu().item())
        
        if val_acc.cpu().item() > ValAccuracy:
            print("Model Re-Saved")
            ValAccuracy = val_acc.cpu().item()
            torch.save(model, ModelName)


    saved_model = torch.load(ModelName, map_location=torch.device('cpu')).to(device)

Train_Accuracy = EvaluateModel(saved_model, Train_DataLoader, device)
print("Train Accuracy = ", Train_Accuracy.cpu().item())
Test_Accuracy = EvaluateModel(saved_model, Test_DataLoader, device)
print("Test Accuracy = ", Test_Accuracy.cpu().item())

print("Generalization Gap = ", (Train_Accuracy.cpu().item() - Test_Accuracy.cpu().item()))

100%|██████████| 141/141 [00:01<00:00, 104.15it/s]


EPOCH -  1 . Train Accuracy =  0.8138039112091064 , Validation Accuracy =  0.8431110978126526
Model Re-Saved


100%|██████████| 141/141 [00:01<00:00, 108.30it/s]


EPOCH -  2 . Train Accuracy =  0.8562549352645874 , Validation Accuracy =  0.8551111221313477
Model Re-Saved


100%|██████████| 141/141 [00:01<00:00, 106.13it/s]


...

EPOCH -  100 . Train Accuracy =  0.9847647547721863 , Validation Accuracy =  0.8844444751739502


100%|██████████| 797/797 [00:07<00:00, 102.33it/s]


Train Accuracy =  0.9824314117431641


100%|██████████| 157/157 [00:01<00:00, 108.19it/s]

Test Accuracy =  0.8844999670982361
Generalization Gap =  0.09793144464492798


In [ ]:
# -------------------------------------------- STL 10 Dataset -----------------------------------------------------------
Option = 3
Train_DataLoader, Val_DataLoader, Test_DataLoader = CreateDataLoaders(Option, 64)
ModelName = "Model_MLP_IOCN_STL10.pt"

model = MLP_IOCN(27648, 10).to(device)
loss_criteria = nn.CrossEntropyLoss()
AdamOpt = torch.optim.Adam(model.parameters(), lr=0.0001)

ValAccuracy = 0

saved = False
if saved == True:
    saved_model = torch.load(ModelName, map_location=torch.device('cpu')).to(device)
else:
    model.train()
    Train_Accuracy = 0

    for e in range(EPOCHS):
        Train_Accuracy = TrainModel(model, loss_criteria, AdamOpt, device, Train_DataLoader)
        val_acc = EvaluateModel(model, Val_DataLoader, device)

        print("EPOCH - ", e+1, ". Train Accuracy = ", Train_Accuracy.cpu().item(), ", Validation Accuracy = ", val_acc.cpu().item())
        
        if val_acc.cpu().item() > ValAccuracy:
            print("Model Re-Saved")
            ValAccuracy = val_acc.cpu().item()
            torch.save(model, ModelName)


    saved_model = torch.load(ModelName, map_location=torch.device('cpu')).to(device)

print()
Train_Accuracy = EvaluateModel(saved_model, Train_DataLoader, device)
print("Train Accuracy = ", Train_Accuracy.cpu().item())
Test_Accuracy = EvaluateModel(saved_model, Test_DataLoader, device)
print("Test Accuracy = ", Test_Accuracy.cpu().item())

print("Generalization Gap = ", (Train_Accuracy.cpu().item() - Test_Accuracy.cpu().item()))

Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 12/12 [00:00<00:00, 23.60it/s]


EPOCH -  1 . Train Accuracy =  0.1576470583677292 , Validation Accuracy =  0.14533333480358124
Model Re-Saved


100%|██████████| 12/12 [00:00<00:00, 35.00it/s]


EPOCH -  2 . Train Accuracy =  0.14541177451610565 , Validation Accuracy =  0.13066665828227997


...

100%|██████████| 12/12 [00:00<00:00, 34.22it/s]


EPOCH -  89 . Train Accuracy =  0.9018823504447937 , Validation Accuracy =  0.30133333802223206


100%|██████████| 12/12 [00:00<00:00, 35.41it/s]


EPOCH -  90 . Train Accuracy =  0.9025882482528687 , Validation Accuracy =  0.30399999022483826


  0%|          | 0/67 [00:00<?, ?it/s]

In [8]:
# -------------------------------------------- CIFAR-10 Dataset -----------------------------------------------------------
Option = 5
Train_DataLoader, Val_DataLoader, Test_DataLoader = CreateDataLoaders(Option, 64)
ModelName = "Model_MLP_IOCN_CIFAR10.pt"

model = MLP_IOCN(3072, 10).to(device)
loss_criteria = nn.CrossEntropyLoss()
AdamOpt = torch.optim.Adam(model.parameters(), lr=0.0001)

ValAccuracy = 0

saved = False
if saved == True:
    saved_model = torch.load(ModelName, map_location=torch.device('cpu')).to(device)
else:
    model.train()
    Train_Accuracy = 0

    for e in range(EPOCHS):
        Train_Accuracy = TrainModel(model, loss_criteria, AdamOpt, device, Train_DataLoader)
        val_acc = EvaluateModel(model, Val_DataLoader, device)

        print("EPOCH - ", e+1, ". Train Accuracy = ", Train_Accuracy.cpu().item(), ", Validation Accuracy = ", val_acc.cpu().item())
        
        if val_acc.cpu().item() > ValAccuracy:
            print("Model Re-Saved")
            ValAccuracy = val_acc.cpu().item()
            torch.save(model, ModelName)

    saved_model = torch.load(ModelName, map_location=torch.device('cpu')).to(device)


print()

Train_Accuracy = EvaluateModel(saved_model, Train_DataLoader, device)
print("Train Accuracy = ", Train_Accuracy.cpu().item())
Test_Accuracy = EvaluateModel(saved_model, Test_DataLoader, device)
print("Test Accuracy = ", Test_Accuracy.cpu().item())

print("Generalization Gap = ", (Train_Accuracy.cpu().item() - Test_Accuracy.cpu().item()))

100%|██████████| 170498071/170498071 [00:04<00:00, 40515881.28it/s]


Extracting ./cifar-10-python.tar.gz to ./
Files already downloaded and verified


100%|██████████| 118/118 [00:01<00:00, 88.33it/s]


EPOCH -  1 . Train Accuracy =  0.3648470640182495 , Validation Accuracy =  0.39399999380111694
Model Re-Saved


100%|██████████| 118/118 [00:01<00:00, 86.81it/s]


...

EPOCH -  100 . Train Accuracy =  0.8922588229179382 , Validation Accuracy =  0.5129333138465881



100%|██████████| 665/665 [00:07<00:00, 88.46it/s]


Train Accuracy =  0.7432470917701721


100%|██████████| 157/157 [00:01<00:00, 87.90it/s]

Test Accuracy =  0.5155999660491943
Generalization Gap =  0.22764712572097778
